## Tool Log Exploratory Data Analysis

### Setup

In [2]:
import pandas as pd
from pathlib import Path

### Read Data

In [3]:
# Set up path to the CSV data in the parent folder's 'data' directory
current_dir = Path.cwd()
data_path = current_dir.parent / "data" / "mock_tool_log_final.csv"

# Load the CSV
df = pd.read_csv(data_path, parse_dates=["start_time", "end_time"])

### Preview Data

In [4]:
df.head()

,tool_id,tool_status,merge_key,start_time,end_time
0,TOOL_001,Scheduled Downtime; Preventive Maintenance; Bi...,1,2024-04-01 00:00:00,2024-04-01 04:00:00
1,TOOL_001,Scheduled Downtime; User Maintenance Delay;,2,2024-04-01 04:00:00,2024-04-01 06:00:00
2,TOOL_001,Scheduled Downtime; Preventive Maintenance; Bi...,3,2024-04-01 06:00:00,2024-04-01 08:00:00
3,TOOL_001,Productive; Engineering Runs; Post Maintenance...,4,2024-04-01 08:00:00,2024-04-01 10:00:00
4,TOOL_001,Productive; Regular Production,5,2024-04-01 10:00:00,2024-04-01 13:00:00


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40982 entries, 0 to 40981
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   tool_id      40982 non-null  object        
 1   tool_status  40982 non-null  object        
 2   merge_key    40982 non-null  int64         
 3   start_time   40982 non-null  datetime64[ns]
 4   end_time     40982 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 1.6+ MB


In [7]:
df.isnull().sum()

tool_id        0
tool_status    0
merge_key      0
start_time     0
end_time       0
dtype: int64

In [6]:
df["tool_status"].unique()

array(['Scheduled Downtime; Preventive Maintenance; Bi-Weekly PM',
       'Scheduled Downtime; User Maintenance Delay;',
       'Productive; Engineering Runs; Post Maintenance Qual',
       'Productive; Regular Production', 'Standby; No Product',
       'Unscheduled Downtime; User Maintenance Delay',
       'Unscheduled Downtime; Repair; Repair - Repair',
       'Standby; No Operator; Waiting For Carrier Delivery/Pickup',
       'Productive; Engineering Runs; Service Routine - Do Not Reload',
       'Productive; Engineering Runs; Processing - Do Not Reload',
       'Productive; Engineering Runs; Holding for Priority Lot',
       'Unscheduled Downtime; Supplier Maintenance Delay; Waiting for Parts (Vendor)'],
      dtype=object)

In [8]:
df.nunique()

tool_id           12
tool_status       12
merge_key      40982
start_time      8735
end_time        8735
dtype: int64